In [1]:
from collections import Counter
from scipy.fftpack import fft    # Fast Fourier Transform
from scipy import stats          # Kernel Density Estimation
import pandas as pd              # datafrme
import numpy as np               # array
from mpl_toolkits.mplot3d import Axes3D # 3D Visualization
import matplotlib.pyplot as plt  # Visualization
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
import sys
import collections
import itertools
from scipy.stats import mode
%matplotlib inline

In [13]:
raw_data = pd.read_csv('final_data.csv')
raw_data.head()

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,...,t493,t494,t495,t496,t497,t498,t499,t500,t501,y
0,0.019336,0.000000,0.000000,0.000000,0.003223,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.029004,0.009668,0.012891,0.000000,0.0,0.0,0.003223,0.003223,0.0,0
1,0.000000,0.000000,0.012891,0.000000,0.016113,0.000000,0.006445,0.000000,0.003223,0.022559,...,0.000000,0.000000,0.000000,0.009668,0.0,0.0,0.000000,0.009668,0.0,0
2,0.000000,0.009668,0.000000,0.000000,0.006445,0.012891,0.000000,0.000000,0.029004,0.025781,...,0.006445,0.003223,0.012891,0.000000,0.0,0.0,0.000000,0.003223,0.0,0
3,0.000000,0.000000,0.000000,0.016113,0.006445,0.003223,0.000000,0.022559,0.012891,0.000000,...,0.000000,0.003223,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0
4,0.016113,0.000000,0.000000,0.000000,0.012891,0.000000,0.000000,0.003223,0.003223,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.003223,0.0,0


In [14]:
#normal_data = raw_data.loc[raw_data['y'] == 0]
#abnormal_data = raw_data.loc[raw_data['y'] == 1]

In [15]:
# 통계치 6개 구해서 RF 돌려보기 or empirical mode decomposition 구해서 RF 돌려보기
# knn, dtw 각각 돌려보기 (1-nn + dtw is best?)

In [16]:
x_train_val, x_test, y_train_val, y_test = train_test_split(raw_data.iloc[:,0:501], raw_data['y'],
                                                            test_size = 0.3, stratify = raw_data['y'], random_state = 7)

x_train, x_val, y_train, y_val = train_test_split(x_train_val.iloc[:,0:501],
                                                  y_train_val, test_size = 0.3, stratify = y_train_val, random_state = 667)

In [ ]:
y_train_val

In [18]:
# split data into train and test sets
#seed = 7
#test_size = 0.3
#x_train, x_test, y_train, y_test = train_test_split(raw_data.iloc[:,0:501], raw_data.iloc[:,501], test_size=test_size, random_state=seed)

In [19]:
for i in range(0, x_train_val.shape[0]):
    x_train_val.iloc[i,:] = (x_train_val.iloc[i,:] - min(x_train_val.iloc[i,:])) / (max(x_train_val.iloc[i,:]) - min(x_train_val.iloc[i,:]))
    
for i in range(0, x_test.shape[0]):
    x_test.iloc[i,:] = (x_test.iloc[i,:] - min(x_test.iloc[i,:])) / (max(x_test.iloc[i,:]) - min(x_test.iloc[i,:]))

In [20]:
try:
    from IPython.display import clear_output
    have_ipython = True
except ImportError:
    have_ipython = False
    
class KnnDtw(object):
    """K-nearest neighbor classifier using dynamic time warping
    as the distance measure between pairs of time series arrays
    
    Arguments
    ---------
    n_neighbors : int, optional (default = 5)
        Number of neighbors to use by default for KNN
        
    max_warping_window : int, optional (default = infinity)
        Maximum warping window allowed by the DTW dynamic
        programming function
            
    subsample_step : int, optional (default = 1)
        Step size for the timeseries array. By setting subsample_step = 2,
        the timeseries length will be reduced by 50% because every second
        item is skipped. Implemented by x[:, ::subsample_step]
    """
    
    def __init__(self, n_neighbors=5, max_warping_window=10000, subsample_step=1):
        self.n_neighbors = n_neighbors
        self.max_warping_window = max_warping_window
        self.subsample_step = subsample_step
    
    def fit(self, x, l):
        """Fit the model using x as training data and l as class labels
        
        Arguments
        ---------
        x : array of shape [n_samples, n_timepoints]
            Training data set for input into KNN classifer
            
        l : array of shape [n_samples]
            Training labels for input into KNN classifier
        """
        
        self.x = x
        self.l = l
        
    def _dtw_distance(self, ts_a, ts_b, d = lambda x,y: abs(x-y)):
        """Returns the DTW similarity distance between two 2-D
        timeseries numpy arrays.

        Arguments
        ---------
        ts_a, ts_b : array of shape [n_samples, n_timepoints]
            Two arrays containing n_samples of timeseries data
            whose DTW distance between each sample of A and B
            will be compared
        
        d : DistanceMetric object (default = abs(x-y))
            the distance measure used for A_i - B_j in the
            DTW dynamic programming function
        
        Returns
        -------
        DTW distance between A and B
        """

        # Create cost matrix via broadcasting with large int
        ts_a, ts_b = np.array(ts_a), np.array(ts_b)
        M, N = len(ts_a), len(ts_b)
        cost = float("inf") * np.ones((M, N))

        # Initialize the first row and column
        cost[0, 0] = d(ts_a[0], ts_b[0])
        for i in range(1, M):
            cost[i, 0] = cost[i-1, 0] + d(ts_a[i], ts_b[0])

        for j in range(1, N):
            cost[0, j] = cost[0, j-1] + d(ts_a[0], ts_b[j])

        # Populate rest of cost matrix within window
        for i in range(1, M):
            for j in range(max(1, i - self.max_warping_window),
                            min(N, i + self.max_warping_window)):
                choices = cost[i - 1, j - 1], cost[i, j-1], cost[i-1, j]
                cost[i, j] = min(choices) + d(ts_a[i], ts_b[j])

        # Return DTW distance given window 
        return cost[-1, -1]
    
    def _dist_matrix(self, x, y):
        """Computes the M x N distance matrix between the training
        dataset and testing dataset (y) using the DTW distance measure
        
        Arguments
        ---------
        x : array of shape [n_samples, n_timepoints]
        
        y : array of shape [n_samples, n_timepoints]
        
        Returns
        -------
        Distance matrix between each item of x and y with
            shape [training_n_samples, testing_n_samples]
        """
        
        # Compute the distance matrix        
        dm_count = 0
        
        # Compute condensed distance matrix (upper triangle) of pairwise dtw distances
        # when x and y are the same array
        if(np.array_equal(x, y)):
            x_s = shape(x)
            dm = np.zeros((x_s[0] * (x_s[0] - 1)) // 2, dtype=np.double)
            
            p = ProgressBar(shape(dm)[0])
            
            for i in range(0, x_s[0] - 1):
                for j in range(i + 1, x_s[0]):
                    dm[dm_count] = self._dtw_distance(x[i, ::self.subsample_step],
                                                      y[j, ::self.subsample_step])
                    
                    dm_count += 1
                    p.animate(dm_count)
            
            # Convert to squareform
            dm = squareform(dm)
            return dm
        
        # Compute full distance matrix of dtw distnces between x and y
        else:
            x_s = np.shape(x)
            y_s = np.shape(y)
            dm = np.zeros((x_s[0], y_s[0])) 
            dm_size = x_s[0]*y_s[0]
            
            p = ProgressBar(dm_size)
        
            for i in range(0, x_s[0]):
                for j in range(0, y_s[0]):
                    dm[i, j] = self._dtw_distance(x[i, ::self.subsample_step],
                                                  y[j, ::self.subsample_step])
                    # Update progress bar
                    dm_count += 1
                    p.animate(dm_count)
        
            return dm
        
    def predict(self, x):
        """Predict the class labels or probability estimates for 
        the provided data

        Arguments
        ---------
          x : array of shape [n_samples, n_timepoints]
              Array containing the testing data set to be classified
          
        Returns
        -------
          2 arrays representing:
              (1) the predicted class labels 
              (2) the knn label count probability
        """
        
        dm = self._dist_matrix(x, self.x)

        # Identify the k nearest neighbors
        knn_idx = dm.argsort()[:, :self.n_neighbors]

        # Identify k nearest labels
        knn_labels = self.l[knn_idx]
        
        # Model Label
        mode_data = mode(knn_labels, axis=1)
        mode_label = mode_data[0]
        mode_proba = mode_data[1]/self.n_neighbors

        return mode_label.ravel(), mode_proba.ravel()

class ProgressBar:
    """This progress bar was taken from PYMC
    """
    def __init__(self, iterations):
        self.iterations = iterations
        self.prog_bar = '[]'
        self.fill_char = '*'
        self.width = 40
        self.__update_amount(0)
        if have_ipython:
            self.animate = self.animate_ipython
        else:
            self.animate = self.animate_noipython

    def animate_ipython(self, iter):
        print('\r', self,
        sys.stdout.flush())
        self.update_iteration(iter + 1)

    def update_iteration(self, elapsed_iter):
        self.__update_amount((elapsed_iter / float(self.iterations)) * 100.0)
        self.prog_bar += '  %d of %s complete' % (elapsed_iter, self.iterations)

    def __update_amount(self, new_amount):
        percent_done = int(round((new_amount / 100.0) * 100.0))
        all_full = self.width - 2
        num_hashes = int(round((percent_done / 100.0) * all_full))
        self.prog_bar = '[' + self.fill_char * num_hashes + ' ' * (all_full - num_hashes) + ']'
        pct_place = (len(self.prog_bar) // 2) - len(str(percent_done))
        pct_string = '%d%%' % percent_done
        self.prog_bar = self.prog_bar[0:pct_place] + \
            (pct_string + self.prog_bar[pct_place + len(pct_string):])

    def __str__(self):
        return str(self.prog_bar)

In [27]:
m = KnnDtw(n_neighbors=1, max_warping_window=2)
#m2 = KnnDtw(n_neighbors=3, max_warping_window=2)
#m3 = KnnDtw(n_neighbors=5, max_warping_window=2)

m.fit(np.array(x_train_val), np.array(y_train_val))
#m2.fit(np.array(x_train_val), np.array(y_train_val))
#m3.fit(np.array(x_train_val), np.array(y_train_val))


In [ ]:
labels = {0:'normal', 1:'abnormal'}
label_1, proba_1 = m.predict(np.array(x_test))
#label_2, proba_2 = m2.predict(np.array(x_test))
#label_3, proba_3 = m3.predict(np.array(x_test))


In [29]:
#print(classification_report(y_test, label, target_names=[l for l in labels.values()]))
print(confusion_matrix(y_test, label_1))
print('acc:', np.mean(y_test == label_1))
print(precision_score(y_test, label_1, pos_label=1))
print(recall_score(y_test, label_1, pos_label=1))
print('f1:', f1_score(y_test, label_1, pos_label=1))

[[17  1]
 [ 1 14]]
acc: 0.9393939393939394
0.9333333333333333
0.9333333333333333
f1: 0.9333333333333333


In [21]:
label_3

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0])

In [22]:
label_2

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0])

In [23]:
proba_3

array([1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.8, 1. ,
       1. , 1. , 1. , 1. , 1. , 0.8, 0.8, 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. ])

In [24]:
proba_2

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.66666667, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        ])

In [69]:
#cross validation
raw_data = pd.read_csv('final_data.csv')

In [35]:
for i in range(0, raw_data.shape[0]):
    raw_data.iloc[i,:-1] = (raw_data.iloc[i,:-1] - min(raw_data.iloc[i,:-1])) / (max(raw_data.iloc[i,:-1]) - min(raw_data.iloc[i,:-1]))

In [70]:
X = np.array(raw_data.iloc[:,:-1])
Y = raw_data.iloc[:,501]

In [71]:
best_model = m2
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=500)

In [ ]:
cv_acc = []
cv_precision = []
cv_recall = []
cv_f1 = []

for train, test in kfold.split(X, Y):

    # evaluate the model
    best_model.fit(np.array(X[train]), np.array(Y[train]))
    label, proba = best_model.predict(X[test])
    
    accuracy = np.mean(label == Y[test].values)
    precision = precision_score(Y[test].values, label, pos_label=1)
    recall = recall_score(Y[test].values, label, pos_label=1)
    f_score = f1_score(Y[test].values, label, pos_label=1)
    
    cv_acc.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    cv_f1.append(f_score)
    
print('accuracy:', np.mean(cv_acc))
print('precision:', np.mean(cv_precision))
print('recall:', np.mean(cv_recall))
print('f1:', np.mean(cv_f1))

In [55]:
cv_precision

[0.7142857142857143,
 0.8333333333333334,
 0.9090909090909091,
 0.7692307692307693,
 0.9090909090909091]

In [56]:
cv_recall

[1.0, 1.0, 1.0, 1.0, 1.0]

In [32]:
from keras.models import load_model
lstm_model = load_model('LSTM_Final_Model_2.h5')

/usr/local/anaconda/envs/env_py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [185]:
x_test_arr = np.array(x_test)
y_pred = lstm_model.predict_classes(x_test_arr.reshape(x_test_arr.shape[0],x_test_arr.shape[1],1))

In [186]:
print(classification_report(y_test, y_pred, target_names=[l for l in labels.values()]))
print(confusion_matrix(y_test, y_pred))
print(np.mean(y_test==y_pred.flatten()))
print(f1_score(y_test, y_pred, pos_label=1))

             precision    recall  f1-score   support

     normal       1.00      1.00      1.00        18
   abnormal       1.00      1.00      1.00        15

avg / total       1.00      1.00      1.00        33

[[18  0]
 [ 0 15]]
1.0
1.0
